In [11]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree, distance_matrix
import networkx as nx


def generate_adjacency_matrices_with_electrode_id(csv_file, threshold_ratio=1.5):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 初始化结果字典
    adjacency_matrices = {}
    layouts = {}

    # 根据label值分组
    for label, group in df.groupby("label"):
        # 提取X, Y, Z坐标和电极编号
        points = group[["X", "Y", "Z"]].values
        electrode_numbers = group["electrode_number"].values

        # 计算所有电极对之间的距离的中位数
        dists = distance_matrix(points, points)
        median_dist = np.median(dists[np.triu_indices_from(dists, k=1)])

        # 使用中位数的一定比例作为阈值
        dynamic_threshold = median_dist * threshold_ratio

        # 构建k-d树
        tree = KDTree(points)

        # 获取电极数量，并以电极编号作为索引创建邻接矩阵
        n = len(electrode_numbers)
        adjacency_matrix = pd.DataFrame(
            data=np.zeros((n, n), dtype=int),
            index=electrode_numbers,
            columns=electrode_numbers,
        )

        # 对每个点查找相邻点
        for i, point in enumerate(points):
            # 查询在动态阈值内的所有点的索引
            indices = tree.query_ball_point(point, r=dynamic_threshold)

            # 更新邻接矩阵，排除自身
            for j in indices:
                if i != j:
                    id_i = electrode_numbers[i]
                    id_j = electrode_numbers[j]
                    adjacency_matrix.at[id_i, id_j] = 1

        # 保存结果
        adjacency_matrices[label] = adjacency_matrix

        # 创建图并查找连通分量
        G = nx.from_pandas_adjacency(adjacency_matrix)
        connected_components = list(nx.connected_components(G))
        layouts[label] = [list(component) for component in connected_components]

    return adjacency_matrices, layouts


# 调用函数并打印结果
adjacency_matrices, layouts = generate_adjacency_matrices_with_electrode_id(
    "/Users/siyuanzhao/Documents/GitHub/CS529_VDS_Epilepsy/frontend/src/data/electrodes/ep129/ep129_rois_kmeans.csv"
)
for label, layout in layouts.items():
    print(f"Label {label} 的电极排列方式和布局 (使用动态阈值):")
    print(layout)

Label 0 的电极排列方式和布局 (使用动态阈值):
[[73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]]
Label 1 的电极排列方式和布局 (使用动态阈值):
[[93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]]
Label 2 的电极排列方式和布局 (使用动态阈值):
[[17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]]


In [13]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
import networkx as nx

def generate_dynamic_layouts(csv_file, threshold_ratio=10):
    # 读取CSV文件
    df = pd.read_csv(csv_file)

    # 初始化结果
    layouts = {}

    # 根据label值分组
    for label, group in df.groupby('label'):
        # 按照电极编号排序
        sorted_group = group.sort_values(by='electrode_number')
        points = sorted_group[['X', 'Y', 'Z']].values
        electrode_numbers = sorted_group['electrode_number'].values

        # 计算距离矩阵并获取动态阈值
        dists = pd.DataFrame(distance_matrix(points, points))
        median_dist = np.median(dists.values[np.triu_indices_from(dists, k=1)])
        dynamic_threshold = median_dist * threshold_ratio

        # 构建k-d树并查询相邻点
        tree = KDTree(points)
        adjacency_matrix = np.zeros((len(electrode_numbers), len(electrode_numbers)), dtype=int)
        for i, point in enumerate(points):
            indices = tree.query_ball_point(point, r=dynamic_threshold)
            for j in indices:
                if i != j:
                    adjacency_matrix[i, j] = 1

        # 使用networkx创建图并查找连通分量
        G = nx.from_numpy_array(adjacency_matrix)
        connected_components = list(nx.connected_components(G))

        # 对每个连通分量，根据电极编号排序来确定布局
        layout = []
        for component in connected_components:
            sorted_component = sorted(list(component), key=lambda x: electrode_numbers[x])
            layout.append([electrode_numbers[i] for i in sorted_component])
        
        layouts[label] = layout

    return layouts

# 调用函数并打印结果
layouts = generate_dynamic_layouts("/Users/siyuanzhao/Documents/GitHub/CS529_VDS_Epilepsy/frontend/src/data/electrodes/ep129/ep129_rois_kmeans.csv")
for label, layout in layouts.items():
    print(f"Label {label} 的电极排列方式和布局 (综合方法):")
    for row in layout:
        print(row)


Label 0 的电极排列方式和布局 (综合方法):
[73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
Label 1 的电极排列方式和布局 (综合方法):
[93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]
Label 2 的电极排列方式和布局 (综合方法):
[17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
